- Be general, oxygenated carbohydrates, see Socrates book on data drive

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Define the alpha value for the post-plasma materials
alpha = 0.65

# Define the color for each material
cb_colors = {
    'γ-Al₂O₃': (0/255, 0/255, 0/255),
    '2% CuO@γ-Al₂O₃': (86/255, 180/255, 233/255),
    '10% CuO@γ-Al₂O₃': (0/255, 114/255, 178/255),
    '2% Fe₂O₃@γ-Al₂O₃': (230/255, 159/255, 0/255),
    '10% Fe₂O₃@γ-Al₂O₃': (213/255, 94/255, 0/255),

    'γ-Al₂O₃ pre-plasma': (
        (0 * alpha + 255 * (1 - alpha)) / 255,
        (0 * alpha + 255 * (1 - alpha)) / 255,
        (0 * alpha + 255 * (1 - alpha)) / 255
    ),
    '2% CuO@γ-Al₂O₃ pre-plasma': (
        (86 * alpha + 255 * (1 - alpha)) / 255,
        (180 * alpha + 255 * (1 - alpha)) / 255,
        (233 * alpha + 255 * (1 - alpha)) / 255
    ),
    '10% CuO@γ-Al₂O₃ pre-plasma': (
        (0 * alpha + 255 * (1 - alpha)) / 255,
        (114 * alpha + 255 * (1 - alpha)) / 255,
        (178 * alpha + 255 * (1 - alpha)) / 255
    ),
    '10% CuO@γ-Al₂O₃\npre-plasma': (
        (0 * alpha + 255 * (1 - alpha)) / 255,
        (114 * alpha + 255 * (1 - alpha)) / 255,
        (178 * alpha + 255 * (1 - alpha)) / 255
    ),
    '2% Fe₂O₃@γ-Al₂O₃ pre-plasma': (
        (230 * alpha + 255 * (1 - alpha)) / 255,
        (159 * alpha + 255 * (1 - alpha)) / 255,
        (0 * alpha + 255 * (1 - alpha)) / 255
    ),
    '10% Fe₂O₃@γ-Al₂O₃ pre-plasma': (
        (213 * alpha + 255 * (1 - alpha)) / 255,
        (94 * alpha + 255 * (1 - alpha)) / 255,
        (0 * alpha + 255 * (1 - alpha)) / 255
    ),
    'γ-Al₂O₃ post-plasma': (0/255, 0/255, 0/255),
    '2% CuO@γ-Al₂O₃ post-plasma': (86/255, 180/255, 233/255),
    '10% CuO@γ-Al₂O₃ post-plasma': (0/255, 114/255, 178/255),
    '10% CuO@γ-Al₂O₃\npost-plasma': (0/255, 114/255, 178/255),
    '2% Fe₂O₃@γ-Al₂O₃ post-plasma': (230/255, 159/255, 0/255),
    '10% Fe₂O₃@γ-Al₂O₃ post-plasma': (213/255, 94/255, 0/255)
}

# determine order of materials for plotting
material_order = [
    'γ-Al₂O₃',
    '2% CuO@γ-Al₂O₃',
    '2% Fe₂O₃@γ-Al₂O₃',
    '10% CuO@γ-Al₂O₃',
    '10% Fe₂O₃@γ-Al₂O₃',
    'γ-Al₂O₃ pre-plasma',
    '2% CuO@γ-Al₂O₃ pre-plasma',
    '2% Fe₂O₃@γ-Al₂O₃ pre-plasma',
    '10% CuO@γ-Al₂O₃ pre-plasma',
    '10% Fe₂O₃@γ-Al₂O₃ pre-plasma',
    'γ-Al₂O₃ post-plasma',
    '2% CuO@γ-Al₂O₃ post-plasma',
    '2% Fe₂O₃@γ-Al₂O₃ post-plasma',
    '10% CuO@γ-Al₂O₃ post-plasma',
    '10% Fe₂O₃@γ-Al₂O₃ post-plasma'
]

In [3]:
# Define the path to the Excel file
excel_file_path = (
    r'N:\FWET\FDCH\AdsCatal\General\personal_work_folders\plasmacatdesign'
    r'\materials\ugent\ugent-ftir.xlsx'
)

# Read all worksheets in the Excel file into a dictionary of DataFrames
data = pd.read_excel(
    io=excel_file_path,
    sheet_name=['spectra', 'peaks']
)

# Assign each DataFrame to a separate variable
spectra_data = data['spectra']
peaks_data = data['peaks']

In [4]:
# The materials of interest
materials_of_interest = [
    'γ-Al₂O₃',
    '2% CuO@γ-Al₂O₃',
    '2% Fe₂O₃@γ-Al₂O₃',
    '10% CuO@γ-Al₂O₃',
    '10% Fe₂O₃@γ-Al₂O₃'
]

# Subset the materials of interest
spectra_subset = spectra_data[
    spectra_data['material'].isin(materials_of_interest)
].copy()

# Subset the most recent measurement for each material
spectra_subset['max_date'] = spectra_subset.groupby(
    ['material', 'plasma']
)['date'].transform(max)

spectra_subset = spectra_subset[
    spectra_subset['date'] == spectra_subset['max_date']
]

spectra_subset = spectra_subset.drop(columns=['max_date'])

# Group the data by material and plasma and calculate a rolling mean
spectra_subset['transmittance_rolling_mean'] = (
    spectra_subset
    .groupby(['material', 'plasma'])['transmittance']
    .transform(lambda x: x.rolling(window=3, center=True).mean())
)

# Create new column material_plasma
spectra_subset['material_plasma'] = (
    spectra_subset['material'] + ' ' + spectra_subset['plasma']
)

In [5]:
# Scale the transmittance data for each material-plasma combination
scaler = MinMaxScaler()
spectra_subset['transmittance_scaled'] = (
    spectra_subset
    .groupby(['material', 'plasma'])['transmittance']
    .transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())
)

In [ ]:
# Use Times New Roman font
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'
plt.rcParams['font.size'] = 7

# Set the size of the figure in inches (1 inch = 25.4 mm)
fig = plt.figure(
    figsize=(90/25.4, 180/25.4),
    dpi=1000
)
gs = gridspec.GridSpec(
    nrows=3,
    ncols=1,
    figure=fig
)

# Create the remaining four subplots
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[2, 0])

# Despine ax1 for the gamma-alumina data
sns.despine(
    ax=ax1,
    top=True, right=True, left=True, bottom=False
)

# Plot the gamma-alumina data
sns.lineplot(
    x='wavenumber',
    y='transmittance_scaled',
    hue='material_plasma',
    data=spectra_subset[spectra_subset['material'] == 'γ-Al₂O₃'],
    palette=cb_colors,
    ax=ax1,
    legend=False,
    linewidth=0.6
)

# Set title for ax1
ax1.set_title('γ-Al₂O₃', fontsize=9)

# ax1 x-axis settings
ax1.set_xlabel('')
ax1.set_xlim(3999, 501)
ax1.spines['bottom'].set_bounds(3999, 501)
ax1.set_xticks(
    ticks=[501, 1000, 1500, 2000, 2500, 3000, 3500, 3999],
    labels=[500, 1000, 1500, 2000, 2500, 3000, 3500, 4000]
)

# ax1 y-axis settings
ax1.set_ylabel('Transmittance (a.u.)', fontsize=8)
ax1.set_ylim(0, 1)
ax1.tick_params(
    axis='y',
    which='both',
    left=False,
    labelleft=False
)

# Add annotation that specifies pre- and post-plasma
ax1.annotate(text='pre-plasma',
             xy=(3500, 0.66),
             xytext=(3600, 0.88),
             textcoords='data',
             fontsize=8,
             color=((0*alpha + 255*(1-alpha))/255,
                    (0*alpha+255*(1-alpha))/255,
                    (0*alpha+255*(1-alpha))/255),
             arrowprops=dict(arrowstyle='->',
                             color=((0*alpha+255*(1-alpha))/255,
                                    (0*alpha+255*(1-alpha))/255,
                                    (0*alpha+255*(1-alpha))/255)
                            )
)

ax1.annotate(
    text='post-plasma',
    xy=(3530, 0.60),
    xytext=(3998, 0.47),
    textcoords='data',
    fontsize=8,
    color='black',
    arrowprops=dict(arrowstyle='->', color='black')
)


# Create a small inset plot, ax1a, for the gamma-alumina data
left, bottom, width, height = [0.21, 0.76, 0.25, 0.11]
ax1a = fig.add_axes([left, bottom, width, height], facecolor='none')
sns.lineplot(
    x='wavenumber',
    y='transmittance_scaled',
    hue='material_plasma',
    hue_order=material_order,
    data=spectra_subset[spectra_subset['material'] == 'γ-Al₂O₃'],
    palette=cb_colors,
    linewidth=.6,
    legend=False,
    ax=ax1a
)

# ax1a x-axis settings
ax1a.set_xlabel(xlabel=None)
ax1a.set_xlim([3000, 2850])
ax1a.set_xticks([2968, 2910])
ax1a.tick_params(
    axis='x',
    colors='dimgrey',
    direction='out',
    labelsize=6,
    length=2,
    pad=2
)
ax1a.spines['bottom'].set_color('dimgrey')
ax1a.spines['bottom'].set_linewidth(0.5)

# ax1a y-axis settings
ax1a.set_ylim([0.72, 0.87])
ax1a.yaxis.set_visible(False)

# despine ax1a
sns.despine(ax=ax1a, top=True, right=True, left=True)

# Create a small inset plot, ax1b, for the gamma-alumina data
left, bottom, width, height = [0.53, 0.76, 0.25, 0.11]
ax1b = fig.add_axes([left, bottom, width, height], facecolor='none')
sns.lineplot(
    x='wavenumber',
    y='transmittance_scaled',
    hue='material_plasma',
    hue_order=material_order,
    data=spectra_subset[spectra_subset['material'] == 'γ-Al₂O₃'],
    palette=cb_colors,
    linewidth=.6,
    legend=False,
    ax=ax1b
)

# ax1b x-axis settings
ax1b.set_xlabel(xlabel=None)
ax1b.set_xlim([1675, 1250])
ax1b.set_xticks([1590, 1462, 1333])
ax1b.tick_params(
    axis='x',
    colors='dimgrey',
    direction='out',
    labelsize=6,
    length=2,
    pad=2
)
ax1b.spines['bottom'].set_color('dimgrey')
ax1b.spines['bottom'].set_linewidth(0.5)

# ax1b y-axis settings
ax1b.set_ylim([0.61, 0.89])
ax1b.yaxis.set_visible(False)

# despine ax1b
sns.despine(ax=ax1b, top=True, right=True, left=True)


# Despine ax2 for the 10% CuO gamma-alumina data
sns.despine(
    ax=ax2,
    top=True, right=True, left=True, bottom=False
)

# Plot the 10% CuO gamma-alumina data
sns.lineplot(
    x='wavenumber',
    y='transmittance_scaled',
    hue='material_plasma',
    data=spectra_subset[spectra_subset['material'] == '10% CuO@γ-Al₂O₃'],
    palette=cb_colors,
    ax=ax2,
    legend=False,
    linewidth=0.6
)

# Set title for ax2
ax2.set_title('10% CuO@γ-Al₂O₃', fontsize=9)

# ax2 x-axis settings
ax2.set_xlabel('')
ax2.set_xlim(3999, 501)
ax2.spines['bottom'].set_bounds(3999, 501)
ax2.set_xticks(
    ticks=[501, 1000, 1500, 2000, 2500, 3000, 3500, 3999],
    labels=[500, 1000, 1500, 2000, 2500, 3000, 3500, 4000]
)

# ax2 y-axis settings
ax2.set_ylabel('Transmittance (a.u.)', fontsize=8)
ax2.set_ylim(0, 1)
ax2.tick_params(
    axis='y',
    which='both',
    left=False,
    labelleft=False
)


# Create a small inset plot, ax2a, for the 10% CuO gamma-alumina data
left, bottom, width, height = [0.21, 0.435, 0.25, 0.11]
ax2a = fig.add_axes([left, bottom, width, height], facecolor='none')
sns.lineplot(
    x='wavenumber',
    y='transmittance_scaled',
    hue='material_plasma',
    hue_order=material_order,
    data=spectra_subset[spectra_subset['material'] == '10% CuO@γ-Al₂O₃'],
    palette=cb_colors,
    linewidth=.6,
    legend=False,
    ax=ax2a
)

# ax2a x-axis settings
ax2a.set_xlabel(xlabel=None)
ax2a.set_xlim([3000, 2850])
ax2a.set_xticks([2968, 2910])
ax2a.tick_params(
    axis='x',
    colors='dimgrey',
    direction='out',
    labelsize=6,
    length=2,
    pad=2
)
ax2a.spines['bottom'].set_color('dimgrey')
ax2a.spines['bottom'].set_linewidth(0.5)

# ax2a y-axis settings
ax2a.set_ylim([0.65, 0.82])
ax2a.yaxis.set_visible(False)

# despine ax2a
sns.despine(ax=ax2a, top=True, right=True, left=True)

# Create a small inset plot, ax2b, for the 10% CuO gamma-alumina data
left, bottom, width, height = [0.53, 0.435, 0.25, 0.11]
ax2b = fig.add_axes([left, bottom, width, height], facecolor='none')
sns.lineplot(
    x='wavenumber',
    y='transmittance_scaled',
    hue='material_plasma',
    hue_order=material_order,
    data=spectra_subset[spectra_subset['material'] == '10% CuO@γ-Al₂O₃'],
    palette=cb_colors,
    linewidth=.6,
    legend=False,
    ax=ax2b
)

# ax2b x-axis settings
ax2b.set_xlabel(xlabel=None)
ax2b.set_xlim([1675, 1250])
ax2b.set_xticks([1590, 1462, 1333])
ax2b.tick_params(
    axis='x',
    colors='dimgrey',
    direction='out',
    labelsize=6,
    length=2,
    pad=2
)
ax2b.spines['bottom'].set_color('dimgrey')
ax2b.spines['bottom'].set_linewidth(0.5)

# ax2b y-axis settings
ax2b.set_ylim([0.49, 0.89])
ax2b.yaxis.set_visible(False)

# despine ax2b
sns.despine(ax=ax2b, top=True, right=True, left=True)


# Despine ax3 for the 10% Fe2O3 gamma-alumina data
sns.despine(
    ax=ax3,
    top=True, right=True, left=True, bottom=False
)

# Plot the 10% Fe2O3 gamma-alumina data
sns.lineplot(
    x='wavenumber',
    y='transmittance_scaled',
    hue='material_plasma',
    data=spectra_subset[spectra_subset['material'] == '10% Fe₂O₃@γ-Al₂O₃'],
    palette=cb_colors,
    ax=ax3,
    legend=False,
    linewidth=0.6
)

# Set title for ax3
ax3.set_title('10% Fe₂O₃@γ-Al₂O₃', fontsize=9)

# ax3 x-axis settings
ax3.set_xlabel('Wavenumber (cm$^{-1}$)', fontsize=8)
ax3.set_xlim(3999, 501)
ax3.spines['bottom'].set_bounds(3999, 501)
ax3.set_xticks(
    ticks=[501, 1000, 1500, 2000, 2500, 3000, 3500, 3999],
    labels=[500, 1000, 1500, 2000, 2500, 3000, 3500, 4000]
)

# ax3 y-axis settings
ax3.set_ylabel('Transmittance (a.u.)', fontsize=8)
ax3.set_ylim(0, 1)
ax3.tick_params(
    axis='y',
    which='both',
    left=False,
    labelleft=False
)


# Create a small inset plot, ax3a, for the 10% Fe2O3 gamma-alumina data
left, bottom, width, height = [0.21, 0.105, 0.25, 0.11]
ax3a = fig.add_axes([left, bottom, width, height], facecolor='none')
sns.lineplot(
    x='wavenumber',
    y='transmittance_scaled',
    hue='material_plasma',
    hue_order=material_order,
    data=spectra_subset[spectra_subset['material'] == '10% Fe₂O₃@γ-Al₂O₃'],
    palette=cb_colors,
    linewidth=.6,
    legend=False,
    ax=ax3a
)

# ax3a x-axis settings
ax3a.set_xlabel(xlabel=None)
ax3a.set_xlim([3000, 2850])
ax3a.set_xticks([2968, 2910])
ax3a.tick_params(
    axis='x',
    colors='dimgrey',
    direction='out',
    labelsize=6,
    length=2,
    pad=2
)
ax3a.spines['bottom'].set_color('dimgrey')
ax3a.spines['bottom'].set_linewidth(0.5)

# ax3a y-axis settings
ax3a.set_ylim([0.62, 0.84])
ax3a.yaxis.set_visible(False)

# despine ax3a
sns.despine(ax=ax3a, top=True, right=True, left=True)


# Create a small inset plot, ax3b, for the 10% Fe2O3 gamma-alumina data
left, bottom, width, height = [0.53, 0.105, 0.25, 0.100]
ax3b = fig.add_axes([left, bottom, width, height], facecolor='none')
sns.lineplot(
    x='wavenumber',
    y='transmittance_scaled',
    hue='material_plasma',
    hue_order=material_order,
    data=spectra_subset[spectra_subset['material'] == '10% Fe₂O₃@γ-Al₂O₃'],
    palette=cb_colors,
    linewidth=.6,
    legend=False,
    ax=ax3b
)

# ax3b x-axis settings
ax3b.set_xlabel(xlabel=None)
ax3b.set_xlim([1675, 1250])
ax3b.set_xticks([1590, 1462, 1333])
ax3b.tick_params(
    axis='x',
    colors='dimgrey',
    direction='out',
    labelsize=6,
    length=2,
    pad=2
)
ax3b.spines['bottom'].set_color('dimgrey')
ax3b.spines['bottom'].set_linewidth(0.5)

# ax3b y-axis settings
ax3b.set_ylim([0.46, 0.89])
ax3b.yaxis.set_visible(False)

# despine ax3b
sns.despine(ax=ax3b, top=True, right=True, left=True)


# # Add lines to visualise the region that is zoomed in on
# Define line segments with start and end points
line_segments = [
    (ax1, [3000, 3000], [0, 0.025]),
    (ax1, [3000, 3485], [0.025, 0.035]),
    (ax1, [3485, 3485], [0.035, 0.12]),
    
    (ax1, [2850, 2850], [0, 0.025]),
    (ax1, [2850, 2460], [0.025, 0.035]),
    (ax1, [2460, 2460], [0.035, 0.12]),
    
    (ax1, [1675, 1675], [0, 0.025]),
    (ax1, [1675, 2175], [0.025, 0.035]),
    (ax1, [2175, 2175], [0.035, 0.12]),
    
    (ax1, [1250, 1250], [0, 0.025]),
    (ax1, [1250, 1140], [0.025, 0.035]),
    (ax1, [1140, 1140], [0.035, 0.12]),
    
    (ax2, [3000, 3000], [0, 0.03]),
    (ax2, [3000, 3485], [0.03, 0.04]),
    (ax2, [3485, 3485], [0.04, 0.13]),
    
    (ax2, [2850, 2850], [0, 0.03]),
    (ax2, [2850, 2460], [0.03, 0.04]),
    (ax2, [2460, 2460], [0.04, 0.13]),
    
    (ax2, [1675, 1675], [0, 0.03]),
    (ax2, [1675, 2175], [0.03, 0.04]),
    (ax2, [2175, 2175], [0.04, 0.13]),
    
    (ax2, [1250, 1250], [0, 0.03]),
    (ax2, [1250, 1140], [0.03, 0.04]),
    (ax2, [1140, 1140], [0.04, 0.13]),
    
    (ax3, [3000, 3000], [0, 0.025]),
    (ax3, [3000, 3485], [0.025, 0.03]),
    (ax3, [3485, 3485], [0.03, 0.12]),
    
    (ax3, [2850, 2850], [0, 0.025]),
    (ax3, [2850, 2460], [0.025, 0.03]),
    (ax3, [2460, 2460], [0.03, 0.12]),
    
    (ax3, [1675, 1675], [0, 0.025]),
    (ax3, [1675, 2175], [0.025, 0.03]),
    (ax3, [2175, 2175], [0.03, 0.12]),
    
    (ax3, [1250, 1250], [0, 0.025]),
    (ax3, [1250, 1140], [0.025, 0.03]),
    (ax3, [1140, 1140], [0.03, 0.12])
]

# Plot each line segment
for plot, x_coords, y_coords in line_segments:
    plot.plot(
        x_coords,
        y_coords,
        color='grey',
        linestyle='--',
        linewidth=0.5,
        alpha=0.9
    )


plt.tight_layout()
plt.savefig(
    fname=r'C:\Users\sbossier\Dropbox\Apps\Overleaf\ladca_incat_envecon_paper_elsevier\figs\sander_ftir.pdf'
)
plt.show()

In [7]:
%reset -f